In [ ]:
# Importing libraries
%matplotlib inline
import numpy                   as np
import pandas                  as pd
import matplotlib              as pl
import seaborn                 as sns
from  sklearn.model_selection  import train_test_split
from  sklearn.tree             import DecisionTreeClassifier
from sklearn.neural_network    import MLPClassifier
from  sklearn.metrics          import accuracy_score
from  sklearn                  import tree

# Análise Exploratória

Vamos começar lendo o conjunto de dados e definindo o nome correto para suas colunas. Além disso, vamos criar dois conjuntos de dado ¨de apoio¨, um contendo apenas variáveis discretas/categóricas e outro com as variáveis contínuas.

In [ ]:
## Reading .csv files
df = pd.read_csv("data/corrected.csv",sep=',',header=None)

## Defining columns
col_names  = pd.read_csv("data/column_names.csv",sep=',',header=None)[0].values
df.columns = col_names


## Identifying columns as categorical or not
categorical_columns = ["protocol_type","service","flag","land","logged_in","root_shell","su_attempted",
                       "is_host_login","is_guest_login","label"]
df_categorical      = df[categorical_columns]
df_non_categorical  = df.drop(categorical_columns, axis=1)

In [ ]:
## A glance at the non categorical data
df_non_categorical.describe()

## Análise de variáveis discretas


### Relação entre Ataques e Serviços de Rede

Vamos começar analisando a relação entre o serviço de rede utilizado pelo destino dos pacotes e os ataques identificados. Para tal, vamos plotar um gráfico de barras que mostra o percentual de participação de cada serviço de rede em cada ataque.

**De modo a facilitar a visualização, vamos considerar apenas variáveis que representem ao menos 1% do percentual total de ataques ou tipos de serviço de rede.**



In [ ]:
## Create a subset of main dataset (f_data) that contains only services and labels that represents at
## least 1% of the total cases

f_services = pd.crosstab(index=df["service"],columns="count")
f_services = f_services/len(df)
f_services = f_services[f_services["count"] > 0.01]

f_attacks = pd.crosstab(index=df["label"],columns="count")
f_attacks = f_attacks/len(df)
f_attacks = f_attacks[f_attacks["count"] > 0.01]

f_data = df[df['service'].isin(list(f_services.index))]
f_data = f_data[f_data['label'].isin(list(f_attacks.index))]

## Create the plot

attack_data = pd.crosstab(index = f_data["label"], columns=f_data["service"])
frequency_table_attack = (attack_data/attack_data.sum())

frequency_table_attack.plot(kind="bar", 
                 figsize=(8,8),
                 stacked=True);

### Relação entre Ataques e Tipos de Protocolo

Utilizando a mesma abordagem anterior, iremos avaliar o percentual de participcação de cada protocolo de rede em cada tipo de ataque.

O objetivo desse análise é visualziar se existe alguma característica que foge do conhecido pela literatura.

In [ ]:
## Create a subset of main dataset (f_data) that contains only protocols and labels that represents at
## least 1% of the total cases

f_protocol = pd.crosstab(index=df["protocol_type"],columns="count")
f_protocol = f_protocol/len(df)
f_protocol = f_protocol[f_protocol["count"] > 0.01]

f_data = df[df['protocol_type'].isin(list(f_protocol.index))]
f_data = f_data[f_data['label'].isin(list(f_attacks.index))]

## Create a Two-Way Table

relationship_protocoal_attack = pd.crosstab(index=f_data["label"], 
                          columns=f_data["protocol_type"])
## Plot the Two-Way Table
relationship_protocoal_attack.plot(kind="bar", 
                 figsize=(8,8),
                 stacked=True);

### Participação percentual de cada tipo de protocolo de rede

O gráfico abaixo visa dar uma visão geral da participação de cada tipo de protocolo na rede na qual os dados foram coletados.

In [ ]:
## Create a cross tab dataframe
protocol_data = pd.crosstab(index = df["protocol_type"],columns="Protocol type")
frequency_table_protocol = (protocol_data/protocol_data.sum())

## Plot the dataframe
frequency_table_protocol.plot.bar();

## Análise de variáveis contínuas

### Histogramas

In [ ]:
df_non_categorical.hist(figsize = (34,40));

### Análise dos usuários logados no sistema

Julgamos interessante avalair o padrão de comportamento dos usuários que de alguma forma estão logados na rede. Dessa forma, iremos criar gráficos que analisam se alguma ameaça foi detectada para os seguintes tipos de usuários: usuários simplesmente logados, usuários logados como host, usuários logados como convidados.

In [ ]:
df_temp = df[["is_guest_login","label"]]
df_temp = df_temp.loc[df_temp['is_guest_login'] == 1]
df_temp.groupby(["label"]).count().plot(kind='bar', title ="Número de registros", figsize=(10, 5), legend=True, fontsize=12)

# Análise de dados redundantes

De acordo com Tavallaee (2009), o grande número de registros redundantes pode enviesar os classificadores para os registros mais frequentes. Desssa forma, o autor propõe um novo dataset, removendo as duplicatas. Dessa forma, analisaremos o dataset KDD99 completo e o sem as duplicatas. Para tal analisaremos a razão entre os dados únicos e os duplicados e plotaremos os gráficos de relação entre os ataques e serviços, e entre os ataques e protocolos utilizados.

In [ ]:
df_temp = df[["is_host_login","label"]]
df_temp = df_temp.loc[df_temp["is_host_login"] == 1]
df_temp.groupby(["label"]).count().plot(kind='bar', title ="Número de registros", figsize=(10, 5), legend=True, fontsize=12)


## Análise de dados redundantes

De acordo com Tavallaee (2009), o grande número de registros redundantes pode enviesar os classificadores para os registros mais frequentes. Desssa forma, o autor propõe um novo dataset, removendo as duplicatas. Dessa forma, analisaremos o dataset KDD99 completo e o sem as duplicatas. Para tal analisaremos a razão entre os dados únicos e os duplicados e plotaremos os gráficos de relação entre os ataques e serviços, e entre os ataques e protocolos utilizados.

In [ ]:
df2 = df.drop_duplicates()
percentage = 1 - float(df2.shape[0])/float(df.shape[0])
print("O dataset sem duplicatas é {}% menor que o dataset original".format(percentage * 100))

# Modelos de classificação

Antes de executar nossos modelos, vamos realizar algumas modificações em nosso conjunto de dados. São elas:

- Transformar categorias que são strings em categorias numéricas 
- Atribuir uma categoria binária de ataque ou não ataque para cada um dos resgitros
- Criar um conjunto de dados para treino e outro para teste e validação dos modelos, usando a proporção (0.8/02)

In [ ]:
# Mapping strings categories to int

df_binary = df.copy()
df_binary.loc[df_binary.label != 'normal.','label']= 1
df_binary.loc[df_binary.label == 'normal.','label']= 0

c1 = df_binary.protocol_type.unique()
c2 = df_binary.service.unique()
c3 = df_binary.flag.unique()

d_protocol = {}
d_service = {}
d_wtver = {}

for i in range(len(c1)):
    d_protocol[c1[i]] = i

for i in range(len(c2)):
    d_service[c2[i]] = i
    
for i in range(len(c3)):
    d_wtver[c3[i]] = i

for i in d_protocol:
    df_binary = df_binary.replace(i, d_protocol[i])
for i in d_service:
    df_binary = df_binary.replace(i, d_service[i])
for i in d_wtver:
    df_binary = df_binary.replace(i, d_wtver[i])
    

import random
random.seed(1610)

## Split the data in train and test datasets

train = df_binary.sample(frac = 0.8,random_state=200)
test  = df_binary.drop(train.index)

X = train.copy()
Y = pd.factorize(X['label'])[0]
X = X.drop('label', 1)


testInput =  test.copy()
correct = pd.factorize(testInput['label'])[0]
testInput = testInput.drop('label', 1)

## Árvore de regressão

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(df.iloc[:,:41], df[41])
p = clf.predict(df.iloc[:,:41])

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier()
randomForest.fit(X,Y)

In [ ]:
# import matplotlib.pyplot as plt

featuresImportance = {}
for i in range(len(df.columns[:len(df.columns)-1])):
    featuresImportance[df.columns[:len(df.columns)-1][i]] = randomForest.feature_importances_[i]

s = [(k, featuresImportance[k]) for k in sorted(featuresImportance, key=featuresImportance.get, reverse=True)]

print("As 5 features mais relevantes para o modelo foram: \n")
for i in range (5):
    print("%s, com influência: %s" % (s[i][0],s[i][1]))


randomForest.score(testInput,correct)

## Support Vector Machine

In [ ]:
from sklearn import svm
SVM = svm.SVC()
SVM.fit(X, Y)

print("A acurácia do modelo foi de: " % (SVM.score(testInput,correct))

## Naives Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X,Y)
print("A acurácia do modelo foi de: " % (gnb.score(testInput,correct))

In [ ]:
# Removing dupicates
df_no_duplicates = df.drop_duplicates()
percentage = 1 - float(df_no_duplicates["protocol_type"].count())/float(df["protocol_type"].count())
print("O dataset sem duplicatas é {}% menor que o dataset original".format(percentage * 100))

In [ ]:
# Extract all strings from columns
protocols = df["protocol_type"].unique()
services  = df["service"].unique()
flags     = df["flag"].unique()

# Mapping strings to int
d_protocol = {}
d_service  = {}
d_flags    = {}
for i in range(len(protocols)):
    d_protocol[protocols[i]] = i

for i in range(len(services)):
    d_service[services[i]] = i
    
for i in range(len(flags)):
    d_flags[flags[i]] = i

# Replace strings in dataframe
for i in d_protocol:
    df = df.replace(i, d_protocol[i])
print("Replaced protocols")
for i in d_service:
    df = df.replace(i, d_service[i])
print("Replaced services")
for i in d_flags:
    df = df.replace(i, d_flags[i])
print("Replaced flags")

In [ ]:
clf = DecisionTreeClassifier()
#clf = clf.fit(df.iloc[:,:41], df["label"])
#p   = clf.predict(df.iloc[:,:41])
cross_val_score(clf, df.iloc[:,:41], df["label"], cv=10)

In [ ]:
success = 0
fail    = 0
for i in range(len(p)):
    if (df["label"][i] == p[i]):
        success += 1
    else:
        fail += 1
print ("Sucess: {}".format(success))
print ("Fail: {}".format(fail))

In [ ]:
df["label"][1]

In [ ]:
p[1]

In [ ]:
X = df.values[:, :41]
Y = df.values[:,41]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)

In [ ]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

In [ ]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

In [ ]:
y_pred_gini = clf_gini.predict(X_test)
y_pred_gini

In [ ]:
y_pred_en = clf_entropy.predict(X_test)
y_pred_en

In [ ]:
print ("Accuracy using Gini Index is {}%".format(accuracy_score(y_test,y_pred_gini)*100))
print ("Accuracy using Entropy is {}%".format(accuracy_score(y_test,y_pred_en)*100))